# 试试

代码来自 `tutorial\6.1_chinese_dataset_uer_t5.py`

In [1]:
import os
import json
import sys

sys.path.append("../")

In [3]:
from openprompt.pipeline_base import PromptForGeneration
from openprompt.prompts.generation_verbalizer import GenerationVerbalizer
from tqdm import tqdm
from openprompt.data_utils import PROCESSORS
import torch
from openprompt.data_utils.utils import InputExample
import argparse
import numpy as np

from openprompt import PromptDataLoader
from openprompt.prompts import ManualVerbalizer
from openprompt.prompts import SoftTemplate
from openprompt import PromptForClassification
import time
import os
import re
from dataclasses import dataclass
from openprompt.utils.crossfit_metrics import evaluate as crossfit_evaluate

In [5]:
@dataclass
class Args:
    max_steps = 5000
    lr = 1e-2
    warmup_step_prompt = 500
    eval_every_steps = 500

args = Args

In [11]:
from transformers import BertTokenizer, MT5ForConditionalGeneration, Text2TextGenerationPipeline, T5ForConditionalGeneration

# 应该用 T5ForConditionalGeneration 的

model_dir = "uer/t5-v1_1-base-chinese-cluecorpussmall"
model_dir = r"G:\code\pretrain_model_dir\t5-base-chinese-cluecorpussmall"

tokenizer = BertTokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
text2text_generator = Text2TextGenerationPipeline(model, tokenizer)
generated_text = text2text_generator("中国的首都是extra0京", max_length=50, do_sample=False)
print(generated_text)

[{'generated_text': 'extra0 北 extra1 extra2 extra3 extra4 extra5 extra6 extra7 extra8 extra9 extra10 extra11 extra12 extra13 extra14 extra15 extra16 extra17 extra18 extra19 extra9'}]


In [12]:
from openprompt.plms.seq2seq import T5TokenizerWrapper


class T5BertTokenizerWrapper(T5TokenizerWrapper):
    def mask_token(self, id):
        return f"extra{id}"

    def mask_token_ids(self, id):
        return self.tokenizer.convert_tokens_to_ids(f"extra{id}")


# 这里将终止符改成了 extra1, 只要输出一个选项就可以了
tokenizer.eos_token = "extra1"
tokenizer_wrapper = T5BertTokenizerWrapper(
    max_seq_length=128, tokenizer=tokenizer, decoder_max_length=5, decode_from_pad=False
)

In [14]:
data_dir = r"G:\dataset\text_classify\诗歌分类"
def load_local_dataset(path="../CCPM/", split="train"):
    import json

    with open(f"{path}/{split}.jsonl", "r", encoding="utf-8") as fin:
        L = fin.readlines()
    data = []
    for json_str in L:
        result = json.loads(json_str)
        # from IPython import embed; embed()
        for idx, choice in enumerate(result["choices"]):
            result[f"choice{idx}"] = choice
        result.pop("choices")
        data.append(InputExample(meta=result, label=int(result["answer"])))
    return data


dataset = {}
dataset["train"] = load_local_dataset(path=data_dir, split="train")
dataset["val"] = load_local_dataset(path=data_dir, split="valid")

In [47]:
dataset["train"][0]

{
  "guid": null,
  "label": 1,
  "meta": {
    "answer": 1,
    "choice0": "行人初上木兰舟",
    "choice1": "骚人遥驻木兰舟",
    "choice2": "有人独上木兰舟",
    "choice3": "行人迢递木兰舟",
    "translation": "诗人啊，你竟像在遥远的地方站立船头。"
  },
  "text_a": "",
  "text_b": "",
  "tgt_text": "二"
}

In [15]:
from openprompt.prompts import ManualTemplate

mytemplate = ManualTemplate(
    tokenizer=tokenizer,
    text="""一：{"meta":"choice0"}； 二： {"meta":"choice1"}； 三： {"meta":"choice2"}； 四：{"meta":"choice3"}。哪一句表达了下面的意思: {"meta":"translation", "shortenable":True, "post_processing": lambda x:x.strip('。')}？第{"mask"}句。 """,
)
myverbalizer = GenerationVerbalizer(tokenizer, label_words={0: "一", 1: "二", 2: "三", 3: "四"}, is_rule=False)
train_dataloader = PromptDataLoader(
    dataset=dataset["train"],
    template=mytemplate,
    verbalizer=myverbalizer,  # be sure to add verbalizer
    tokenizer_wrapper=tokenizer_wrapper,
    batch_size=16,
    shuffle=True,
    teacher_forcing=True,
    predict_eos_token=True,
    truncate_method="tail",
)
validation_dataloader = PromptDataLoader(
    dataset=dataset["val"],
    template=mytemplate,
    verbalizer=myverbalizer,
    tokenizer=tokenizer,
    tokenizer_wrapper=tokenizer_wrapper,
    batch_size=16,
    shuffle=False,
    teacher_forcing=False,
    predict_eos_token=False,  # predict_eos_token=True or False are both ok
    truncate_method="tail",
)


tokenizing: 21778it [00:19, 1128.57it/s]
tokenizing: 2720it [00:02, 1183.25it/s]

[CLS] 一 ： 寥 寥 清 磬 闻 ； 二 ： 寥 寥 竟 不 闻 ； 三 ： 寥 寥 不 复 闻 ； 四 ： 寥 寥 不 可 闻 。 哪 一 句 表 达 了 下 面 的 意 思 : 后 来 几 乎 闻 所 未 闻 了 ？ 第 extra0 句 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
extra0 四 [PAD] [PAD] [PAD]


In [16]:
# visualize some encoded data
print(tokenizer.decode(next(iter(train_dataloader))["input_ids"][0]))
print(tokenizer.decode(next(iter(train_dataloader))["decoder_input_ids"][0]))

[CLS] 一 ： 明 日 知 何 处 ； 二 ： 近 日 何 多 念 ； 三 ： 明 日 何 其 多 ； 四 ： 明 日 复 何 之 。 哪 一 句 表 达 了 下 面 的 意 思 : 明 天 何 等 的 多 ？ 第 extra0 句 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
extra0 四 [PAD] [PAD] [PAD]


In [17]:
use_cuda = True
prompt_model = PromptForGeneration(plm=model, template=mytemplate, freeze_plm=False, plm_eval_mode=False)
if use_cuda:
    prompt_model = prompt_model.cuda()

print("truncate rate: {}".format(train_dataloader.tokenizer_wrapper.truncate_rate), flush=True)

truncate rate: 0.02559392603477835


In [29]:
generation_arguments = {
    "max_new_tokens": 1,
}

def evaluate(prompt_model, dataloader):
    """
    评估方法
    """
    predictions = []
    ground_truths = []

    for step, inputs in enumerate(dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        _, output_sentence = prompt_model.generate(inputs, **generation_arguments, verbose=False)
        predictions.extend(output_sentence)
        ground_truths.extend(inputs["tgt_text"])
    assert len(predictions) == len(ground_truths), (len(predictions), len(ground_truths))
    predictions = [prediction.strip("extra0 ") for prediction in predictions]
    ground_truths = [ground_truth.strip() for ground_truth in ground_truths]
    # shown one example
    print(f"predictions {predictions[0]}, ground_truths {ground_truths[0]}")
    score = sum(
        [
            prediction[: len(ground_truth)] == ground_truth
            for prediction, ground_truth in zip(predictions, ground_truths)
        ]
    ) / len(ground_truths)
    return score


In [30]:
from torch.optim import AdamW
from transformers import (
    get_linear_schedule_with_warmup,
    get_constant_schedule_with_warmup,
)  # use AdamW is a standard practice for transformer
from transformers.optimization import Adafactor  # use Adafactor is the default setting for T5

loss_func = torch.nn.CrossEntropyLoss()

tot_step = args.max_steps


# normally we freeze the model when using soft_template. However, we keep the option to tune plm
no_decay = ["bias", "LayerNorm.weight"]  # it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters1 = [
    {
        "params": [p for n, p in prompt_model.plm.named_parameters() if (not any(nd in n for nd in no_decay))],
        "weight_decay": 0.01,
    },
    {
        "params": [p for n, p in prompt_model.plm.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer1 = AdamW(optimizer_grouped_parameters1, lr=args.lr)
scheduler1 = get_linear_schedule_with_warmup(optimizer1, num_warmup_steps=500, num_training_steps=tot_step)

In [31]:
tot_loss = 0
log_loss = 0
best_val_acc = 0
glb_step = 0
actual_step = 0
leave_training = False
gradient_accumulation_steps = 1

acc_traces = []
tot_train_time = 0
pbar_update_freq = 50
prompt_model.train()

PromptForGeneration(
  (prompt_model): PromptModel(
    (plm): T5ForConditionalGeneration(
      (shared): Embedding(21228, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(21228, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(in_features=768, out_features=768, bias=False)
                  (k): Linear(in_features=768, out_features=768, bias=False)
                  (v): Linear(in_features=768, out_features=768, bias=False)
                  (o): Linear(in_features=768, out_features=768, bias=False)
                  (relative_attention_bias): Embedding(32, 12)
                )
                (layer_norm): T5LayerNorm()
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (1): T5LayerFF(
                (DenseReluDense): T5DenseActDense(
                  (wi): Linear(in_featu

In [32]:
pbar = tqdm(total=tot_step, desc="Train")
for epoch in range(1000000):
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        tot_train_time -= time.time()
        loss = prompt_model(inputs)
        loss.backward()
        tot_loss += loss.item()
        actual_step += 1

        if actual_step % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(prompt_model.parameters(), 1.0)
            glb_step += 1
            if glb_step % pbar_update_freq == 0:
                aveloss = (tot_loss - log_loss) / pbar_update_freq
                pbar.update(pbar_update_freq)
                pbar.set_postfix({"loss": aveloss, "epoch": epoch})
                log_loss = tot_loss

                if optimizer1 is not None:
                    optimizer1.step()
                    optimizer1.zero_grad()
                if scheduler1 is not None:
                    scheduler1.step()

        tot_train_time += time.time()

        if actual_step % gradient_accumulation_steps == 0 and glb_step > 0 and glb_step % args.eval_every_steps == 0:
            val_acc = evaluate(prompt_model, validation_dataloader)
            if val_acc >= best_val_acc:
                # torch.save(prompt_model.state_dict(),f"{args.project_root}/../ckpts/{this_run_unicode}.ckpt")
                best_val_acc = val_acc

            acc_traces.append(val_acc)
            print(
                "Glb_step {}, val_acc {}, average time {}".format(glb_step, val_acc, tot_train_time / actual_step),
                flush=True,
            )
            prompt_model.train()

        if glb_step > args.max_steps:
            leave_training = True
            break

    if leave_training:
        break


Train:  71%|███████   | 3550/5000 [11:32<04:42,  5.13it/s, loss=1.85, epoch=2]


predictions 三, ground_truths 四
Glb_step 500, val_acc 0.25919117647058826, average time 0.13028615856170656


predictions 三, ground_truths 四
Glb_step 1000, val_acc 0.24632352941176472, average time 0.13074163126945496


predictions 三, ground_truths 四
Glb_step 1500, val_acc 0.3775735294117647, average time 0.1308801472981771


predictions 三, ground_truths 四
Glb_step 2000, val_acc 0.6209558823529412, average time 0.13095311641693116


...

predictions 三, ground_truths 四
Glb_step 3000, val_acc 0.7470588235294118, average time 0.1313468804359436


predictions 三, ground_truths 四
Glb_step 3500, val_acc 0.7794117647058824, average time 0.13135348987579346


predictions 四, ground_truths 四
Glb_step 4000, val_acc 0.7919117647058823, average time 0.1313677254319191


predictions 四, ground_truths 四
Glb_step 4500, val_acc 0.8180147058823529, average time 0.1314443656073676


predictions 三, ground_truths 四
Glb_step 5000, val_acc 0.7610294117647058, average time 0.13144027066230773


# 预测

In [35]:
example = next(iter(validation_dataloader))
print(example)
print(example.keys())

{"input_ids": [[101, 671, 8038, 3934, 4128, 4127, 1908, 3209, 8039, 753, 8038, 3655, 4128, 4127, 1348, 4197, 8039, 676, 8038, 3655, 4128, 3266, 1908, 3209, 8039, 1724, 8038, 3655, 4128, 4127, 1348, 3209, 511, 1525, 671, 1368, 6134, 6809, 749, 678, 7481, 4638, 2692, 2590, 131, 3210, 3266, 4638, 4128, 4219, 4127, 749, 1348, 6158, 7028, 3173, 4157, 778, 8043, 5018, 21128, 1368, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 671, 8038, 674, 7027, 7474, 3702, 1812, 8039, 753, 8038, 674, 7027, 5318, 3702, 1812, 8039, 676, 8038, 674, 7027, 5318, 7599, 4170, 8039, 1724, 8038, 674, 7027, 5318, 1973, 3702, 511, 1525, 671, 1368, 6134, 6809, 749, 678, 7481, 4638, 2692, 2590, 131, 3882, 5782, 674, 7027, 8024, 2014, 1399, 6823, 2813, 8043, 5018, 21128, 1368, 511, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [37]:
print(example["tgt_text"])
print(example["input_ids"].shape)

['四', '四', '三', '四', '三', '三', '二', '四', '二', '一', '三', '一', '二', '二', '三', '一']
torch.Size([16, 128])


In [46]:
# 预测
prompt_model.eval()
with torch.no_grad():
    if use_cuda:
        example = example.cuda()
    _, output_sentence = prompt_model.generate(inputs, **generation_arguments, verbose=False)
print(output_sentence)
print(example["tgt_text"])

['三 三', '一 一', '三 三', '四 四', '一 一', '一 一', '二 二', '三 一', '二 二', '三 三', '三 二', '二 二', '四 四', '四 四', '四 二', '一 三']
['四', '四', '三', '四', '三', '三', '二', '四', '二', '一', '三', '一', '二', '二', '三', '一']


In [41]:
evaluate(prompt_model, validation_dataloader)

predictions 三, ground_truths 四


0.8330882352941177

In [50]:
data_dir = r"G:\dataset\text_classify\诗歌分类"
def load_local_dataset(path="../CCPM/", split="train"):
    import json

    with open(f"{path}/{split}.jsonl", "r", encoding="utf-8") as fin:
        L = fin.readlines()
    data = []
    for json_str in L:
        result = json.loads(json_str)
        # from IPython import embed; embed()
        for idx, choice in enumerate(result["choices"]):
            result[f"choice{idx}"] = choice
        result.pop("choices")
        if "answer" not in result:
            label = None
        else:
            label = int(result["answer"])

        data.append(InputExample(meta=result, label=label))
    return data


dataset["test"] = load_local_dataset(path=data_dir, split="test_public")

In [52]:
dataset["test"][0]

{
  "guid": null,
  "label": null,
  "meta": {
    "choice0": "事主不尽年",
    "choice1": "事君期尽年",
    "choice2": "事主不及终",
    "choice3": "万年不尽意",
    "translation": "侍奉夫主，不能尽自己的天年。"
  },
  "text_a": "",
  "text_b": "",
  "tgt_text": null
}

# TODO: 不能直接用 PromptDataLoader, 这个要求有 label

In [53]:
from openprompt.prompts import ManualTemplate

mytemplate = ManualTemplate(
    tokenizer=tokenizer,
    text="""一：{"meta":"choice0"}； 二： {"meta":"choice1"}； 三： {"meta":"choice2"}； 四：{"meta":"choice3"}。哪一句表达了下面的意思: {"meta":"translation", "shortenable":True, "post_processing": lambda x:x.strip('。')}？第{"mask"}句。 """,
)
myverbalizer = GenerationVerbalizer(tokenizer, label_words={0: "一", 1: "二", 2: "三", 3: "四"}, is_rule=False)

test_dataloader = PromptDataLoader(
    dataset=dataset["test"],
    template=mytemplate,
    # 原来预测的时候不加这个就行了
    # verbalizer=myverbalizer,
    tokenizer=tokenizer,
    tokenizer_wrapper=tokenizer_wrapper,
    batch_size=16,
    shuffle=False,
    teacher_forcing=False,
    predict_eos_token=True,  # predict_eos_token=True or False are both ok
    truncate_method="tail",
)


tokenizing: 0it [00:00, ?it/s]

tokenizing: 2720it [00:02, 1170.55it/s]


In [54]:
# visualize some encoded data
print(tokenizer.decode(next(iter(test_dataloader))["input_ids"][0]))
print(tokenizer.decode(next(iter(test_dataloader))["decoder_input_ids"][0]))

[CLS] 一 ： 事 主 不 尽 年 ； 二 ： 事 君 期 尽 年 ； 三 ： 事 主 不 及 终 ； 四 ： 万 年 不 尽 意 。 哪 一 句 表 达 了 下 面 的 意 思 : 侍 奉 夫 主 ， 不 能 尽 自 己 的 天 年 ？ 第 extra0 句 。 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
extra0 [PAD] [PAD] [PAD] [PAD]


In [56]:
predictions = []

for step, inputs in enumerate(test_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    _, output_sentence = prompt_model.generate(inputs, **generation_arguments, verbose=False)
    predictions.extend(output_sentence)
predictions = [prediction.strip("extra0 ") for prediction in predictions]
# shown one example
print(f"predictions {predictions[0]}")

predictions 一


In [59]:
len(predictions), len(test_dataloader.raw_dataset)

(2720, 2720)

In [60]:
predictions[:10]

['一', '一', '二', '二', '一', '四', '一', '二', '二', '二']

In [61]:
test_dataloader.raw_dataset[0]

{
  "guid": null,
  "label": null,
  "meta": {
    "choice0": "事主不尽年",
    "choice1": "事君期尽年",
    "choice2": "事主不及终",
    "choice3": "万年不尽意",
    "translation": "侍奉夫主，不能尽自己的天年。"
  },
  "text_a": "",
  "text_b": "",
  "tgt_text": null
}

In [62]:
# 保存结果
label_maps = {
    "一": 0,
    "二": 1,
    "三": 2,
    "四": 3,
}

with open("CCPM.jsonl", "w", encoding="utf-8") as fw, open(os.path.join(data_dir, "test_public.jsonl"), "r", encoding="utf-8") as fr:
    lines = fr.readlines()
    for line, predict in zip(lines, predictions):
        result = json.loads(line)
        result["answer"] = label_maps[predict]
        fw.write(json.dumps(result, ensure_ascii=False) + "\n")

# 评测成绩 81.65

排行榜上的第一位是 93.82, 差距还是非常大.